In [ ]:
import numpy as np
import csv
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import u_V
from random import uniform
from time import time
import gc
import os

# Add progress bar support with fallback
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable, *args, **kwargs):
        return iterable

logger = Logging.setup_logging()

BATCH_SIZE = 5000  # Number of MOSFETs per run
PARAMS_FILE = 'mosfet_params_v9.csv'
MEASUREMENTS_FILE = 'measurements_v9.csv'

FIXED_LAMBDA = 0.02  # Typical channel-length modulation parameter
FIXED_PHI = 0.4  # More realistic Fermi potential

def generate_random_level1_model():
    return {
        'W'     : uniform(10e-6, 100e-6),  # Channel width (1µm - 50µm)
        'L'     : uniform(1e-6, 10e-6),  # Channel length (2µm - 10µm)
        'KP'    : uniform(50e-6, 500e-6),  # Transconductance parameter (50µA/V² - 500µA/V²)
        'VTO'   : uniform(0.3, 1.2),  # Threshold voltage (0.3V - 1.2V)
        'LAMBDA': FIXED_LAMBDA,  # Fixed channel-length modulation
        'PHI'   : FIXED_PHI  # More realistic Fermi potential (0.4V instead of 0.7V)
    }

def build_nmos_circuit(model_params):
    circuit = Circuit('Random_Level1_NMOS_Test')
    circuit.model('NMOSMOD', 'NMOS', LEVEL=1, **model_params)
    circuit.MOSFET('M1', 'drain', 'gate', 'source', 'source', model='NMOSMOD')
    circuit.V('dd', 'drain', circuit.gnd, 0@u_V)
    circuit.R('SRC', 'source', circuit.gnd, 0)
    circuit.V('in', 'gate', circuit.gnd, 0@u_V)
    return circuit

def get_output_characteristic(model_params, gate_voltages, vds_start=0.0, vds_stop=5.0, vds_step=0.1):
    circuit = build_nmos_circuit(model_params)
    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    results = []
    meas_index = 0  # Initialize measurement index

    for vgs in gate_voltages:
        circuit['Vin'].dc_value = vgs
        analysis = simulator.dc(Vdd=slice(vds_start, vds_stop, vds_step))
        vds_array = np.array(analysis.nodes['drain'])
        id_array  = -np.array(analysis.branches['vdd'])
        
        # Append measurement index to each data point
        for vds, id_val in zip(vds_array, id_array):
            results.append((meas_index, vgs, vds, id_val))
            meas_index += 1  # Increment measurement index

    return results

def get_last_mosfet_id():
    if not os.path.exists(PARAMS_FILE):
        return 0  # No previous data exists
    with open(PARAMS_FILE, 'r') as file:
        lines = file.readlines()
        if len(lines) <= 1:
            return 0
        return int(lines[-1].split(',')[0])

def main():
    start_time = time()
    gate_voltages = np.arange(0.0, 5.5, 0.5)
    last_id = get_last_mosfet_id()
    print(f"Last MOSFET ID: {last_id}")
    
    # Open CSV files in append mode
    with open(PARAMS_FILE, 'a', newline='') as params_file, \
         open(MEASUREMENTS_FILE, 'a', newline='') as measurements_file:
        
        params_writer = csv.writer(params_file)
        measurements_writer = csv.writer(measurements_file)
        
        # If the files are new, write headers
        if last_id == 0:
            params_writer.writerow(["MOSFET_ID", "L", "W", "KP", "VTO", "LAMBDA", "PHI"])
            measurements_writer.writerow(["MOSFET_ID", "meas_index", "VGS", "VDS", "ID"])

        mosfet_iterator = tqdm(range(last_id + 1, last_id + BATCH_SIZE + 1), 
                               desc="Generating MOSFETs", 
                               total=BATCH_SIZE, 
                               unit="device", 
                               dynamic_ncols=True)
        
        try:
            for mosfet_id in mosfet_iterator:
                model_params = generate_random_level1_model()
                
                # Write parameters immediately
                params_writer.writerow([
                    mosfet_id,
                    model_params['L'],
                    model_params['W'],
                    model_params['KP'],
                    model_params['VTO'],
                    model_params['LAMBDA'],  # Fixed value
                    model_params['PHI']
                ])
                
                # Generate and write measurements
                measurements = get_output_characteristic(
                    model_params,
                    gate_voltages=gate_voltages
                )
                
                for meas_index, vgs, vds, id_val in measurements:
                    measurements_writer.writerow([mosfet_id, meas_index, vgs, vds, id_val])
                
                # Flush periodically
                if mosfet_id % 100 == 0:
                    params_file.flush()
                    measurements_file.flush()
                    gc.collect()
            
        except KeyboardInterrupt:
            print("\nProcess interrupted. Partial data saved.")
    
    total_time = time() - start_time
    print(f"Completed {BATCH_SIZE} MOSFETs in {total_time:.2f}s")
    print(f"Rate: {BATCH_SIZE/total_time:.2f} devices/sec")

if __name__ == "__main__":
    main()


Last MOSFET ID: 45000


Generating MOSFETs: 100%|██████████| 5000/5000 [02:35<00:00, 32.20device/s]

Completed 5000 MOSFETs in 155.36s
Rate: 32.18 devices/sec
